## PASCAL VOC 2007

| **Loss**          | **IoU50**  | **IoU75**  | **IoU50:95** | **PIoU50** | **PIoU75** | **PIoU50:95** |
| ----------------  | ---------- | ---------- | ------------ | ---------- | ---------- | ------------- |
| ProbIoU           | 0.4035     | 0.0632     | 0.1433       | 0.5136     | 0.2858     | 0.2861        |
| GIoU              | XXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXXXXX |
| DIoU              | XXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXXXXX |
| CIoU              | XXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXX | XXXXXXXXXXXXX |
| IoU (default)     | 0.5250     | 0.1334     | 0.2156       | 0.6322     | 0.4174     | 0.3840        |

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import keras.backend as K
from utils.eval import *
from generators.voc_generator import PascalVocGenerator
from utils.image import preprocess_image
import models

Using TensorFlow backend.


In [2]:
common_args = {
    'batch_size': 1,
    'image_min_side': 256,
    'image_max_side': 512,
    'preprocess_image': preprocess_image,
}
generator = PascalVocGenerator(
    '/datasets/dataset/VOCdevkit/VOC2007',
    'test',
    shuffle_groups=False,
    skip_truncated=False,
    skip_difficult=True,
    **common_args
)

In [3]:
model_path = './snapshots/pascal_resnet50_probiou/finetuned/resnet50_pascal.h5'
# load retinanet model
K.clear_session()
K.set_learning_phase(1)
model = models.load_model(model_path, backbone_name='resnet50')
# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

W0504 19:12:24.270689 140688801236800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0504 19:12:24.271656 140688801236800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0504 19:12:24.284628 140688801236800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0504 19:12:24.293321 140688801236800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W050

In [4]:
average_precisions = evaluate_mAP(generator, model, method='iou')

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('IoU mAP0.5:0.95 = {:.4f}'.format(mean_ap))

Running network: 100% (4952 of 4952) |###| Elapsed Time: 0:16:02 Time:  0:16:02
Parsing annotations: 100% (4952 of 4952) || Elapsed Time: 0:00:01 Time:  0:00:01
Evaluating threshold 0.50: 100% (20 of 20) || Elapsed Time: 0:00:19 Time:  0:00:19
Evaluating threshold 0.55: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

AP0.50 = 0.4035


Evaluating threshold 0.55: 100% (20 of 20) || Elapsed Time: 0:00:19 Time:  0:00:19
Evaluating threshold 0.60: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

AP0.55 = 0.3446


Evaluating threshold 0.60: 100% (20 of 20) || Elapsed Time: 0:00:19 Time:  0:00:19
Evaluating threshold 0.65: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

...

AP0.85 = 0.0074


Evaluating threshold 0.90: 100% (20 of 20) || Elapsed Time: 0:00:19 Time:  0:00:19
Evaluating threshold 0.95: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

AP0.90 = 0.0015


Evaluating threshold 0.95: 100% (20 of 20) || Elapsed Time: 0:00:19 Time:  0:00:19


AP0.95 = 0.0004
num_fp=3614742.0, num_tp=40178.0
285 instances of class aeroplane with average precision: 0.1407
337 instances of class bicycle with average precision: 0.1995
459 instances of class bird with average precision: 0.1183
263 instances of class boat with average precision: 0.0635
469 instances of class bottle with average precision: 0.0528
213 instances of class bus with average precision: 0.1751
1201 instances of class car with average precision: 0.2167
358 instances of class cat with average precision: 0.2264
756 instances of class chair with average precision: 0.0700
244 instances of class cow with average precision: 0.1417
206 instances of class diningtable with average precision: 0.1105
489 instances of class dog with average precision: 0.1867
348 instances of class horse with average precision: 0.1489
325 instances of class motorbike with average precision: 0.1628
4528 instances of class person with average precision: 0.1483
480 instances of class pottedplant with ave

In [5]:
average_precisions = evaluate_mAP(generator, model, method='piou')

# compute per class average precision
total_instances = []
precisions = []
for label, (average_precision, num_annotations) in average_precisions.items():
    print('{:.0f} instances of class'.format(num_annotations), generator.label_to_name(label),
          'with average precision: {:.4f}'.format(average_precision))
    total_instances.append(num_annotations)
    precisions.append(average_precision)
mean_ap = sum(precisions) / sum(x > 0 for x in total_instances)
print('ProbIoU mAP0.5:0.95 = {:.4f}'.format(mean_ap))

Running network: 100% (4952 of 4952) |###| Elapsed Time: 0:16:00 Time:  0:16:00
Parsing annotations: 100% (4952 of 4952) || Elapsed Time: 0:00:01 Time:  0:00:01
Evaluating threshold 0.50: 100% (20 of 20) || Elapsed Time: 0:03:16 Time:  0:03:16
Evaluating threshold 0.55: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

AP0.50 = 0.5136


Evaluating threshold 0.55: 100% (20 of 20) || Elapsed Time: 0:03:39 Time:  0:03:39
Evaluating threshold 0.60: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

AP0.55 = 0.4988


Evaluating threshold 0.60: 100% (20 of 20) || Elapsed Time: 0:03:19 Time:  0:03:19
Evaluating threshold 0.65: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

...

AP0.85 = 0.0784


Evaluating threshold 0.90: 100% (20 of 20) || Elapsed Time: 0:03:08 Time:  0:03:08
Evaluating threshold 0.95: N/A% (0 of 20) || Elapsed Time: 0:00:00 ETA:  --:--:--

AP0.90 = 0.0170


Evaluating threshold 0.95: 100% (20 of 20) || Elapsed Time: 0:03:52 Time:  0:03:52


AP0.95 = 0.0010
num_fp=3587280.0, num_tp=67640.0
285 instances of class aeroplane with average precision: 0.2979
337 instances of class bicycle with average precision: 0.3697
459 instances of class bird with average precision: 0.2810
263 instances of class boat with average precision: 0.1686
469 instances of class bottle with average precision: 0.1214
213 instances of class bus with average precision: 0.3280
1201 instances of class car with average precision: 0.3926
358 instances of class cat with average precision: 0.4186
756 instances of class chair with average precision: 0.1499
244 instances of class cow with average precision: 0.2822
206 instances of class diningtable with average precision: 0.2198
489 instances of class dog with average precision: 0.3609
348 instances of class horse with average precision: 0.3256
325 instances of class motorbike with average precision: 0.3204
4528 instances of class person with average precision: 0.3147
480 instances of class pottedplant with ave